# Algorithm selection

In this script, we train many "quick and dirty" classifiers with default parameters and choose the best one for further optimization.

To run this script, the following data files are needed:
- dt_final.xlsx (attached)

In [2]:
# data
import pandas as pd
import numpy as np
import csv

# misc
import os
import re
import time
import ast
import warnings
import math
import copy
import matplotlib.pyplot as plt

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn import metrics
import scikitplot as skplt

from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
import statistics
from tensorflow.keras import layers

In [3]:

data = pd.read_excel('data/dt_final.xlsx',engine='openpyxl')

## 1 Data preparation

In [4]:
dt = data[[
 'label4',
 'glove_vec300_norm', # normalize
 'tfidf_art', # normalize
 'negative_conc',
 'positive_conc',
 'weak_subj',
 'strong_subj',
 'hyperbolic_terms',
 'attitude_markers',
 'kill_verbs',
 'bias_lexicon',
 'assertive_verbs',
 'factive_verbs',
 'report_verbs',
 'implicative_verbs',
 'hedges',
 'boosters',
 'affect ',
 'posemo ',
 'negemo ',
 'anx ',
 'anger ',
 'sad ',
 'social ',
 'family ',
 'friend ',
 'female ',
 'male ',
 'cogproc ',
 'insight ',
 'cause ',
 'discrep ',
 'tentat ',
 'certain ',
 'differ ',
 'percept ',
 'see ',
 'hear ',
 'feel ',
 'bio ',
 'body ',
 'health ',
 'sexual ',
 'ingest ',
 'drives ',
 'affiliation ',
 'achieve ',
 'power ',
 'reward ',
 'risk ',
 'focuspast ',
 'focuspresent ',
 'focusfuture ',
 'relativ ',
 'motion ',
 'space ',
 'time ',
 'work ',
 'leisure ',
 'home ',
 'money ',
 'relig ',
 'death ',
 'informal ',
 'swear ',
 'netspeak ',
 'assent ',
 'nonflu ',
 'filler ',
 'pos_ADJ',
 'pos_ADP',
 'pos_ADV',
 'pos_AUX',
 'pos_DET',
 'pos_INTJ',
 'pos_NOUN',
 'pos_PRON',
 'pos_PROPN',
 'pos_SCONJ',
 'pos_VERB',
 'pos_X',
 'dep_ROOT',
 'dep_acl',
 'dep_acomp',
 'dep_advcl',
 'dep_advmod',
 'dep_agent',
 'dep_amod',
 'dep_appos',
 'dep_attr',
 'dep_aux',
 'dep_auxpass',
 'dep_case',
 'dep_cc',
 'dep_ccomp',
 'dep_compound',
 'dep_conj',
 'dep_csubj',
 'dep_dative',
 'dep_dep',
 'dep_det',
 'dep_dobj',
 'dep_expl',
 'dep_intj',
 'dep_mark',
 'dep_meta',
 'dep_neg',
 'dep_nmod',
 'dep_npadvmod',
 'dep_nsubj',
 'dep_nsubjpass',
 'dep_nummod',
 'dep_oprd',
 'dep_parataxis',
 'dep_pcomp',
 'dep_pobj',
 'dep_poss',
 'dep_preconj',
 'dep_predet',
 'dep_prep',
 'dep_prt',
 'dep_punct',
 'dep_quantmod',
 'dep_relcl',
 'dep_xcomp',
 'ne_CARDINAL',
 'ne_DATE',
 'ne_EVENT',
 'ne_FAC',
 'ne_GPE',
 'ne_LANGUAGE',
 'ne_LAW',
 'ne_LOC',
 'ne_MONEY',
 'ne_NORP',
 'ne_ORDINAL',
 'ne_ORG',
 'ne_PERCENT',
 'ne_PERSON',
 'ne_PRODUCT',
 'ne_QUANTITY',
 'ne_TIME',
 'ne_WORK_OF_ART',
 'negative_conc_context',
 'positive_conc_context',
 'weak_subj_context',
 'strong_subj_context',
 'hyperbolic_terms_context',
 'attitude_markers_context',
 'kill_verbs_context',
 'bias_lexicon_context',
 'assertive_verbs_context',
 'factive_verbs_context',
 'report_verbs_context',
 'implicative_verbs_context',
 'hedges_context',
 'boosters_context',
 'affect _context',
 'posemo _context',
 'negemo _context',
 'anx _context',
 'anger _context',
 'sad _context',
 'social _context',
 'family _context',
 'friend _context',
 'female _context',
 'male _context',
 'cogproc _context',
 'insight _context',
 'cause _context',
 'discrep _context',
 'tentat _context',
 'certain _context',
 'differ _context',
 'percept _context',
 'see _context',
 'hear _context',
 'feel _context',
 'bio _context',
 'body _context',
 'health _context',
 'sexual _context',
 'ingest _context',
 'drives _context',
 'affiliation _context',
 'achieve _context',
 'power _context',
 'reward _context',
 'risk _context',
 'focuspast _context',
 'focuspresent _context',
 'focusfuture _context',
 'relativ _context',
 'motion _context',
 'space _context',
 'time _context',
 'work _context',
 'leisure _context',
 'home _context',
 'money _context',
 'relig _context',
 'death _context',
 'informal _context',
 'swear _context',
 'netspeak _context',
 'assent _context',
 'nonflu _context',
 'filler _context',
 'pos_ADJ_context',
 'pos_ADP_context',
 'pos_ADV_context',
 'pos_AUX_context',
 'pos_DET_context',
 'pos_INTJ_context',
 'pos_NOUN_context',
 'pos_PRON_context',
 'pos_PROPN_context',
 'pos_SCONJ_context',
 'pos_VERB_context',
 'pos_X_context',
 'dep_ROOT_context',
 'dep_acl_context',
 'dep_acomp_context',
 'dep_advcl_context',
 'dep_advmod_context',
 'dep_agent_context',
 'dep_amod_context',
 'dep_appos_context',
 'dep_attr_context',
 'dep_aux_context',
 'dep_auxpass_context',
 'dep_case_context',
 'dep_cc_context',
 'dep_ccomp_context',
 'dep_compound_context',
 'dep_conj_context',
 'dep_csubj_context',
 'dep_dative_context',
 'dep_dep_context',
 'dep_det_context',
 'dep_dobj_context',
 'dep_expl_context',
 'dep_intj_context',
 'dep_mark_context',
 'dep_meta_context',
 'dep_neg_context',
 'dep_nmod_context',
 'dep_npadvmod_context',
 'dep_nsubj_context',
 'dep_nsubjpass_context',
 'dep_nummod_context',
 'dep_oprd_context',
 'dep_parataxis_context',
 'dep_pcomp_context',
 'dep_pobj_context',
 'dep_poss_context',
 'dep_preconj_context',
 'dep_predet_context',
 'dep_prep_context',
 'dep_prt_context',
 'dep_punct_context',
 'dep_quantmod_context',
 'dep_relcl_context',
 'dep_xcomp_context',
 'ne_CARDINAL_context',
 'ne_DATE_context',
 'ne_EVENT_context',
 'ne_FAC_context',
 'ne_GPE_context',
 'ne_LAW_context',
 'ne_LOC_context',
 'ne_MONEY_context',
 'ne_NORP_context',
 'ne_ORDINAL_context',
 'ne_ORG_context',
 'ne_PERCENT_context',
 'ne_PERSON_context',
 'ne_PRODUCT_context',
 'ne_QUANTITY_context',
 'ne_TIME_context',
 'ne_WORK_OF_ART_context',
 'ne_LANGUAGE_context']]

In [5]:
print('Number of missing values:')
mis = (dt.isnull().sum()).to_frame()
for i, row in mis.iterrows():
    if row[0] > 0:
        print(i, row[0])

# Delete the rows with missing article text:
dt = dt.dropna(subset=['tfidf_art'])

Number of missing values:
tfidf_art 654


In [6]:
x = dt.drop(['label4'], 1)
y4 = dt[['label4']]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12332\2443302479.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = dt.drop(['label4'], 1)


In [7]:
x_scaled = copy.deepcopy(x)

scaler = MinMaxScaler(feature_range=(0, 1))
x_scaled['glove_vec300_norm'] = scaler.fit_transform(x_scaled[['glove_vec300_norm']])
x_scaled['tfidf_art'] = scaler.fit_transform(x_scaled[['tfidf_art']])

In [8]:
train_features, test_features, train_labels, test_labels = train_test_split(x, y4, 
                                                                            test_size = 0.20, random_state = 42)
train_features_sc, test_features_sc, train_labels, test_labels = train_test_split(x_scaled, y4, 
                                                                            test_size = 0.20, random_state = 42)

In [9]:
feature_list = list(x.columns)
len(feature_list)

282

In [10]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (24712, 282)
Training Labels Shape: (24712, 1)
Testing Features Shape: (6178, 282)
Testing Labels Shape: (6178, 1)


## 2 Select the algorithm among many classic classifiers

By default 1 is the positive class, 0 is the negative class
- High precision relates to a low false positive rate
- High recall relates to a low false negative rate.

In [11]:
sorted(metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [12]:
scoring = ['accuracy', 'f1', 'roc_auc', 'precision', 'recall']

In [13]:
print('Biased words:',round(len(y4[y4['label4']==1])/len(y4)*100,0),'%')
print('Biased words:',round(len(y4[y4['label4']==0])/len(y4)*100,0),'%')

Biased words: 10.0 %
Biased words: 90.0 %


### 2.1 Logistic regression

In [14]:
logreg_w = LogisticRegressionCV(random_state=42, class_weight = {0:10, 1:90})
scores_logreg_w = cross_validate(logreg_w, x_scaled, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iteration

In [15]:
print("F1: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_f1'].mean(), scores_logreg_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_precision'].mean(), scores_logreg_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_recall'].mean(), scores_logreg_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_roc_auc'].mean(), scores_logreg_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_logreg_w['test_accuracy'].mean(), scores_logreg_w['test_accuracy'].std() * 2))

F1: 0.38 (+/- 0.06)
Precision: 0.26 (+/- 0.05)
Recall: 0.67 (+/- 0.07)
AUC: 0.82 (+/- 0.04)
Accuracy: 0.79 (+/- 0.05)


In [16]:
pred_logreg_w_cm = logreg_w.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_logreg_w_cm = pred_logreg_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_logreg_w_cm).transpose())

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iteration

Confusion matrix:
 [[4402  195]
 [1177  404]]


### 2.2 LDA

In [17]:
lda = LinearDiscriminantAnalysis()
scores_lda = cross_validate(lda, x_scaled, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

In [18]:
print("F1: %0.2f (+/- %0.2f)" % (scores_lda['test_f1'].mean(), scores_lda['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_lda['test_precision'].mean(), scores_lda['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_lda['test_recall'].mean(), scores_lda['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_lda['test_roc_auc'].mean(), scores_lda['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_lda['test_accuracy'].mean(), scores_lda['test_accuracy'].std() * 2))

F1: 0.41 (+/- 0.06)
Precision: 0.50 (+/- 0.08)
Recall: 0.35 (+/- 0.07)
AUC: 0.82 (+/- 0.04)
Accuracy: 0.90 (+/- 0.01)


In [19]:
pred_lda_cm = lda.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_lda_cm = pred_lda_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_lda_cm).transpose())

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[5375  401]
 [ 204  198]]


### 2.3 QDA

In [20]:
qda = QuadraticDiscriminantAnalysis()
scores_qda = cross_validate(qda, x_scaled, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d ar

In [21]:
print("F1: %0.2f (+/- %0.2f)" % (scores_qda['test_f1'].mean(), scores_qda['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_qda['test_precision'].mean(), scores_qda['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_qda['test_recall'].mean(), scores_qda['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_qda['test_roc_auc'].mean(), scores_qda['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_qda['test_accuracy'].mean(), scores_qda['test_accuracy'].std() * 2))

F1: 0.19 (+/- 0.00)
Precision: 0.10 (+/- 0.00)
Recall: 0.98 (+/- 0.02)
AUC: 0.77 (+/- 0.02)
Accuracy: 0.18 (+/- 0.02)


In [22]:
pred_qda_cm = qda.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_qda_cm = pred_qda_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_qda_cm).transpose())

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[ 533    7]
 [5046  592]]


c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


### 2.4 Naive Bayes

In [23]:
nb = GaussianNB()
scores_nb = cross_validate(nb, x_scaled, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

In [24]:
print("F1: %0.2f (+/- %0.2f)" % (scores_nb['test_f1'].mean(), scores_nb['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_nb['test_precision'].mean(), scores_nb['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_nb['test_recall'].mean(), scores_nb['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_nb['test_roc_auc'].mean(), scores_nb['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_nb['test_accuracy'].mean(), scores_nb['test_accuracy'].std() * 2))

F1: 0.20 (+/- 0.01)
Precision: 0.11 (+/- 0.00)
Recall: 0.98 (+/- 0.02)
AUC: 0.59 (+/- 0.03)
Accuracy: 0.24 (+/- 0.03)


In [25]:
pred_nb_cm = nb.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_nb_cm = pred_nb_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_nb_cm).transpose())

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[ 931   10]
 [4648  589]]


In [26]:
nb_imb = ComplementNB()
scores_nb_imb = cross_validate(nb_imb, x_scaled, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

In [27]:
print("F1: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_f1'].mean(), scores_nb_imb['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_precision'].mean(), scores_nb_imb['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_recall'].mean(), scores_nb_imb['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_roc_auc'].mean(), scores_nb_imb['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_nb_imb['test_accuracy'].mean(), scores_nb_imb['test_accuracy'].std() * 2))

F1: 0.36 (+/- 0.04)
Precision: 0.23 (+/- 0.03)
Recall: 0.74 (+/- 0.04)
AUC: 0.82 (+/- 0.03)
Accuracy: 0.74 (+/- 0.05)


In [28]:
pred_nb_imb_cm = nb_imb.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_nb_imb_cm = pred_nb_imb_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_nb_imb_cm).transpose())

Confusion matrix:
 [[4084  163]
 [1495  436]]


c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### 2.5 KNN

In [29]:
knn = KNeighborsClassifier()
scores_knn = cross_validate(knn, x_scaled, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

In [30]:
print("F1: %0.2f (+/- %0.2f)" % (scores_knn['test_f1'].mean(), scores_knn['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_knn['test_precision'].mean(), scores_knn['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_knn['test_recall'].mean(), scores_knn['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_knn['test_roc_auc'].mean(), scores_knn['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_knn['test_accuracy'].mean(), scores_knn['test_accuracy'].std() * 2))

F1: 0.21 (+/- 0.03)
Precision: 0.45 (+/- 0.08)
Recall: 0.14 (+/- 0.03)
AUC: 0.69 (+/- 0.02)
Accuracy: 0.90 (+/- 0.01)


In [31]:
pred_knn_cm = knn.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_knn_cm = pred_knn_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_knn_cm).transpose())

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Confusion matrix:
 [[5481  515]
 [  98   84]]


### 2.6 Decision Tree

In [32]:
dt = DecisionTreeClassifier(random_state=42)
scores_dt = cross_validate(dt, x, y4, cv=10, scoring=scoring)

In [33]:
print("F1: %0.2f (+/- %0.2f)" % (scores_dt['test_f1'].mean(), scores_dt['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_dt['test_precision'].mean(), scores_dt['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_dt['test_recall'].mean(), scores_dt['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_dt['test_roc_auc'].mean(), scores_dt['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_dt['test_accuracy'].mean(), scores_dt['test_accuracy'].std() * 2))

F1: 0.32 (+/- 0.05)
Precision: 0.31 (+/- 0.06)
Recall: 0.33 (+/- 0.06)
AUC: 0.63 (+/- 0.03)
Accuracy: 0.86 (+/- 0.02)


In [34]:
pred_dt_cm = dt.fit(train_features, train_labels).predict(test_features)
pred_binary_dt_cm = pred_dt_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_dt_cm).transpose())

Confusion matrix:
 [[5122  395]
 [ 457  204]]


In [35]:
dt_w = DecisionTreeClassifier(random_state=42, class_weight = {0:10, 1:90})
scores_dt_w = cross_validate(dt_w, x, y4, cv=10, scoring=scoring)

In [36]:
print("F1: %0.2f (+/- %0.2f)" % (scores_dt_w['test_f1'].mean(), scores_dt_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_dt_w['test_precision'].mean(), scores_dt_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_dt_w['test_recall'].mean(), scores_dt_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_dt_w['test_roc_auc'].mean(), scores_dt_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_dt_w['test_accuracy'].mean(), scores_dt_w['test_accuracy'].std() * 2))

F1: 0.32 (+/- 0.05)
Precision: 0.31 (+/- 0.04)
Recall: 0.33 (+/- 0.06)
AUC: 0.63 (+/- 0.03)
Accuracy: 0.86 (+/- 0.01)


In [37]:
pred_dt_w_cm = dt_w.fit(train_features, train_labels).predict(test_features)
pred_binary_dt_w_cm = pred_dt_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_dt_w_cm).transpose())

Confusion matrix:
 [[5135  388]
 [ 444  211]]


### 2.7 Random Forest

In [38]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)
scores_rf = cross_validate(rf, x, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector 

In [39]:
print("F1: %0.2f (+/- %0.2f)" % (scores_rf['test_f1'].mean(), scores_rf['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_rf['test_precision'].mean(), scores_rf['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_rf['test_recall'].mean(), scores_rf['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_rf['test_roc_auc'].mean(), scores_rf['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf['test_accuracy'].mean(), scores_rf['test_accuracy'].std() * 2))

F1: 0.28 (+/- 0.05)
Precision: 0.74 (+/- 0.11)
Recall: 0.17 (+/- 0.04)
AUC: 0.83 (+/- 0.03)
Accuracy: 0.91 (+/- 0.00)


In [40]:
pred_rf_cm = rf.fit(train_features, train_labels).predict(test_features)
pred_binary_rf_cm = pred_rf_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_rf_cm).transpose())

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12332\3680289860.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  pred_rf_cm = rf.fit(train_features, train_labels).predict(test_features)


Confusion matrix:
 [[5541  495]
 [  38  104]]


In [41]:
rf_w = RandomForestClassifier(random_state=42, class_weight = {0:10, 1:90}, n_estimators=100)
scores_rf_w = cross_validate(rf_w, x, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector 

In [42]:
print("F1: %0.2f (+/- %0.2f)" % (scores_rf_w['test_f1'].mean(), scores_rf_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_rf_w['test_precision'].mean(), scores_rf_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_rf_w['test_recall'].mean(), scores_rf_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_rf_w['test_roc_auc'].mean(), scores_rf_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_rf_w['test_accuracy'].mean(), scores_rf_w['test_accuracy'].std() * 2))

F1: 0.27 (+/- 0.04)
Precision: 0.72 (+/- 0.11)
Recall: 0.17 (+/- 0.03)
AUC: 0.84 (+/- 0.03)
Accuracy: 0.91 (+/- 0.00)


In [43]:
pred_rf_w_cm = rf_w.fit(train_features, train_labels).predict(test_features)
pred_binary_rf_w_cm = pred_rf_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_rf_w_cm).transpose())

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_12332\4262172450.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  pred_rf_w_cm = rf_w.fit(train_features, train_labels).predict(test_features)


Confusion matrix:
 [[5534  495]
 [  45  104]]


### 2.8 SVM (linear kernel)

In [44]:
svm = SVC(kernel='linear', random_state=42)
scores_svm = cross_validate(svm, x_scaled, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

In [45]:
print("F1: %0.2f (+/- %0.2f)" % (scores_svm['test_f1'].mean(), scores_svm['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_svm['test_precision'].mean(), scores_svm['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_svm['test_recall'].mean(), scores_svm['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_svm['test_roc_auc'].mean(), scores_svm['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_svm['test_accuracy'].mean(), scores_svm['test_accuracy'].std() * 2))

F1: 0.19 (+/- 0.09)
Precision: 0.60 (+/- 0.15)
Recall: 0.11 (+/- 0.06)
AUC: 0.69 (+/- 0.07)
Accuracy: 0.91 (+/- 0.00)


In [46]:
pred_svm_cm = svm.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_svm_cm = pred_svm_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_svm_cm).transpose())

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[5531  538]
 [  48   61]]


In [47]:
start_time = time.time()
svm_w = SVC(kernel='linear', random_state=42, class_weight = {0:10, 1:90})
scores_svm_w = cross_validate(svm_w, x_scaled, y4, cv=5, scoring=scoring)
end_time = time.time()
print('Time to train weighted linear SVM on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

Time to train weighted linear SVM on all features, 10-fold CV: 15296.43 seconds


In [48]:
print("F1: %0.2f (+/- %0.2f)" % (scores_svm_w['test_f1'].mean(), scores_svm_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_svm_w['test_precision'].mean(), scores_svm_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_svm_w['test_recall'].mean(), scores_svm_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_svm_w['test_roc_auc'].mean(), scores_svm_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_svm_w['test_accuracy'].mean(), scores_svm_w['test_accuracy'].std() * 2))

F1: 0.38 (+/- 0.04)
Precision: 0.26 (+/- 0.04)
Recall: 0.70 (+/- 0.05)
AUC: 0.83 (+/- 0.02)
Accuracy: 0.78 (+/- 0.05)


In [49]:
start_time = time.time()
pred_svm_w_cm = svm_w.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_svm_w_cm = pred_svm_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_svm_w_cm).transpose())
end_time = time.time()
print('Time to train weighted linear SVM on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[4363  171]
 [1216  428]]
Time to train weighted linear SVM on all features, 10-fold CV: 2975.92 seconds


### 2.9 SVM (non-linear kernel)

In [50]:
start_time = time.time()
svm_w = SVC(random_state=42, class_weight = {0:10, 1:90})
scores_svm_w = cross_validate(svm_w, x_scaled, y4, cv=5, scoring=scoring)
end_time = time.time()
print('Time to train weighted linear SVM on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

Time to train weighted linear SVM on all features, 10-fold CV: 2179.32 seconds


In [51]:
print("F1: %0.2f (+/- %0.2f)" % (scores_svm_w['test_f1'].mean(), scores_svm_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_svm_w['test_precision'].mean(), scores_svm_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_svm_w['test_recall'].mean(), scores_svm_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_svm_w['test_roc_auc'].mean(), scores_svm_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_svm_w['test_accuracy'].mean(), scores_svm_w['test_accuracy'].std() * 2))

F1: 0.34 (+/- 0.02)
Precision: 0.39 (+/- 0.04)
Recall: 0.30 (+/- 0.04)
AUC: 0.78 (+/- 0.02)
Accuracy: 0.89 (+/- 0.01)


In [52]:
start_time = time.time()
pred_svm_w_cm = svm_w.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_svm_w_cm = pred_svm_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_svm_w_cm).transpose())
end_time = time.time()
print('Time to train weighted linear SVM on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[5276  405]
 [ 303  194]]
Time to train weighted linear SVM on all features, 10-fold CV: 414.64 seconds


### 2.10 XGBoost

In [53]:
xgb = xgboost.XGBClassifier(objective="binary:logistic", random_state=42)
scores_xgb = cross_validate(xgb, x, y4, cv=10, scoring=scoring)

In [54]:
print("F1: %0.2f (+/- %0.2f)" % (scores_xgb['test_f1'].mean(), scores_xgb['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_xgb['test_precision'].mean(), scores_xgb['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_xgb['test_recall'].mean(), scores_xgb['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_xgb['test_roc_auc'].mean(), scores_xgb['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_xgb['test_accuracy'].mean(), scores_xgb['test_accuracy'].std() * 2))

F1: 0.35 (+/- 0.05)
Precision: 0.61 (+/- 0.08)
Recall: 0.25 (+/- 0.05)
AUC: 0.85 (+/- 0.03)
Accuracy: 0.91 (+/- 0.01)


In [55]:
pred_xgb_cm = xgb.fit(train_features, train_labels).predict(test_features)
pred_binary_xgb_cm = pred_xgb_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_xgb_cm).transpose())

Confusion matrix:
 [[5502  448]
 [  77  151]]


In [56]:
scale_pos_weight = len(y4[y4['label4']==0])/len(y4[y4['label4']==1])
scale_pos_weight

9.46054859464951

In [57]:
xgb_w = xgboost.XGBClassifier(objective="binary:logistic", scale_pos_weight=scale_pos_weight, random_state=42)
scores_xgb_w = cross_validate(xgb_w, x, y4, cv=10, scoring=scoring)

In [58]:
print("F1: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_f1'].mean(), scores_xgb_w['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_precision'].mean(), scores_xgb_w['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_recall'].mean(), scores_xgb_w['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_roc_auc'].mean(), scores_xgb_w['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_xgb_w['test_accuracy'].mean(), scores_xgb_w['test_accuracy'].std() * 2))

F1: 0.43 (+/- 0.04)
Precision: 0.32 (+/- 0.04)
Recall: 0.64 (+/- 0.08)
AUC: 0.84 (+/- 0.04)
Accuracy: 0.84 (+/- 0.03)


In [59]:
pred_xgb_w_cm = xgb_w.fit(train_features, train_labels).predict(test_features)
pred_binary_xgb_w_cm = pred_xgb_w_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_xgb_w_cm).transpose())

Confusion matrix:
 [[4748  215]
 [ 831  384]]


### 2.11 MLP

In [60]:
mlp = MLPClassifier(random_state=42)
scores_mlp = cross_validate(mlp, x_scaled, y4, cv=10, scoring=scoring)

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1091: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1091: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimizati

In [61]:
print("F1: %0.2f (+/- %0.2f)" % (scores_mlp['test_f1'].mean(), scores_mlp['test_f1'].std() * 2))
print("Precision: %0.2f (+/- %0.2f)" % (scores_mlp['test_precision'].mean(), scores_mlp['test_precision'].std() * 2))
print("Recall: %0.2f (+/- %0.2f)" % (scores_mlp['test_recall'].mean(), scores_mlp['test_recall'].std() * 2))
print("AUC: %0.2f (+/- %0.2f)" % (scores_mlp['test_roc_auc'].mean(), scores_mlp['test_roc_auc'].std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_mlp['test_accuracy'].mean(), scores_mlp['test_accuracy'].std() * 2))

F1: 0.33 (+/- 0.06)
Precision: 0.36 (+/- 0.06)
Recall: 0.30 (+/- 0.08)
AUC: 0.77 (+/- 0.02)
Accuracy: 0.88 (+/- 0.01)


In [62]:
pred_mlp_cm = mlp.fit(train_features_sc, train_labels).predict(test_features_sc)
pred_binary_mlp_cm = pred_mlp_cm.round()
print('Confusion matrix:\n', metrics.confusion_matrix(test_labels, pred_binary_mlp_cm).transpose())

c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1091: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Confusion matrix:
 [[5287  407]
 [ 292  192]]


c:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [63]:
def define_model(n_input):
    # define model
    mlp_w = Sequential()
    # define first hidden layer and visible layer
    mlp_w.add(layers.Dense(100, input_dim=n_input, activation='relu'))
    # define output layer
    mlp_w.add(layers.Dense(1, activation='sigmoid'))
    # define loss and optimizer
    mlp_w.compile(loss='binary_crossentropy', optimizer='adam', metrics=['AUC','Precision','Recall'])
    return mlp_w

In [64]:
# K-fold Cross Validation model evaluation
start_time = time.time()

num_folds = 10
fold_no = 1
F1_per_fold = []
precision_per_fold = []
recall_per_fold = []
AUC_per_fold = []

kf = KFold(n_splits=10, random_state=42, shuffle=True)

x_scaled_np = x_scaled.to_numpy()
y4_np = y4.to_numpy()

for train_index, test_index in kf.split(x_scaled_np):
    X_train, X_test = x_scaled_np[train_index], x_scaled_np[test_index]
    y_train, y_test = y4_np[train_index], y4_np[test_index]

    # Generate a print
    print(f'Training for fold {fold_no} ...')
    
    mlp_w = define_model(len(feature_list))

    # Fit data to model
    mlp_w.fit(X_train, y_train, epochs=200, class_weight={0:10, 1:90})
    mlp_w_pred = mlp_w.predict(X_test)
    mlp_w_pred_bin = mlp_w_pred.round()
    
    F1_per_fold.append(metrics.f1_score(y_test, mlp_w_pred_bin, pos_label=1))
    precision_per_fold.append(metrics.precision_score(y_test, mlp_w_pred_bin, pos_label=1))
    recall_per_fold.append(metrics.recall_score(y_test, mlp_w_pred_bin, pos_label=1))
    AUC_per_fold.append(metrics.roc_auc_score(y_test, mlp_w_pred_bin))
    print('CF:', metrics.confusion_matrix(y_test, mlp_w_pred_bin).transpose())

    # Increase fold number
    fold_no = fold_no + 1

end_time = time.time()
print('Time to train weighted MLP on all features, 10-fold CV:', round((end_time-start_time),2), 'seconds')

Training for fold 1 ...
Epoch 1/200
869/869 [==============================] - 7s 2ms/step - loss: 9.1952 - auc: 0.8151 - precision: 0.2327 - recall: 0.7085
Epoch 2/200
869/869 [==============================] - 2s 2ms/step - loss: 8.3992 - auc: 0.8499 - precision: 0.2580 - recall: 0.7508
Epoch 3/200
869/869 [==============================] - 2s 2ms/step - loss: 8.0303 - auc: 0.8643 - precision: 0.2687 - recall: 0.7738
Epoch 4/200
869/869 [==============================] - 2s 2ms/step - loss: 7.6499 - auc: 0.8791 - precision: 0.2827 - recall: 0.7983
Epoch 5/200
869/869 [==============================] - 2s 2ms/step - loss: 7.2456 - auc: 0.8930 - precision: 0.3004 - recall: 0.8161
Epoch 6/200
869/869 [==============================] - 2s 2ms/step - loss: 6.8031 - auc: 0.9072 - precision: 0.3204 - recall: 0.8357
Epoch 7/200
869/869 [==============================] - 2s 2ms/step - loss: 6.3377 - auc: 0.9212 - precision: 0.3407 - recall: 0.8652
Epoch 8/200
869/869 [========================

In [65]:
print("F1: %0.2f (+/- %0.2f)" % (statistics.mean(F1_per_fold), statistics.pstdev(F1_per_fold) * 2))
print("Precision: %0.2f (+/- %0.2f)" % (statistics.mean(precision_per_fold), statistics.pstdev(precision_per_fold) * 2))
print("Recall: %0.2f (+/- %0.2f)" % (statistics.mean(recall_per_fold), statistics.pstdev(recall_per_fold) * 2))
print("AUC: %0.2f (+/- %0.2f)" % (statistics.mean(AUC_per_fold), statistics.pstdev(AUC_per_fold) * 2))

F1: 0.34 (+/- 0.04)
Precision: 0.34 (+/- 0.06)
Recall: 0.35 (+/- 0.04)
AUC: 0.64 (+/- 0.02)
